In [ ]:
ruta_input = "../data/"
import pandas as pd
df = pd.read_csv(ruta_input+'Detalles_clientes.csv',index_col=False)

In [ ]:
print(df.columns)
df.info()

In [ ]:
df['valor'] = df['valor'].astype(str).str.replace("'", "")
df['valor'] = pd.to_numeric(df['valor'], errors='coerce')
df['clase'] = df['clase'].astype('category')
df['fecha_efectiva'] = pd.to_datetime(df['fecha_efectiva'], format='%Y%m%d')

In [ ]:
# agregar la frecuencia de compras por cliente
df['frecuencia_compras'] = df.groupby('Id_cliente')['Id_cliente'].transform('count')
df

In [ ]:
# histograma para 'frecuecia_compras'
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.hist(df['frecuencia_compras'], bins=4, color='blue', alpha=0.7)
plt.title('Histograma de Frecuencia de Compras por Cliente')
plt.xlabel('Frecuencia de Compras')
plt.ylabel('Número de Clientes')
plt.grid(axis='y', alpha=0.75)
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# --- Configuración de Estilo para los Gráficos ---
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['axes.labelsize'] = 12

print("Iniciando Fase 1: Análisis Exploratorio de Datos (EDA) 📊")
print("==========================================================")


# --- 1. Análisis Estadístico General ---
print("\n### 1. Análisis Estadístico General ###")

# Clientes y transacciones totales
num_clientes = df['Id_cliente'].nunique()
num_transacciones = len(df)
print(f"Número de clientes únicos: {num_clientes}")
print(f"Número de transacciones totales: {num_transacciones}")

# Estadísticas descriptivas de la columna 'valor'
print("\nEstadísticas del valor de las transacciones:")
print(df['valor'].describe().apply('{:,.2f}'.format))
print("----------------------------------------------------------")


# --- 2. Análisis por Tipo de Transacción (clase) ---
print("\n### 2. Análisis por Tipo de Transacción (COMPRA vs AVANCE) ###")

# Conteo y valor por clase
clase_counts = df['clase'].value_counts()
clase_valor = df.groupby('clase')['valor'].sum()

print("\nDistribución de transacciones por clase:")
print(clase_counts)
print("\nValor total por clase:")
print(clase_valor.apply('{:,.2f}'.format))

# Gráfico 1: Cantidad de transacciones por clase
plt.figure()
ax = sns.countplot(x='clase', data=df, palette='viridis', order=clase_counts.index)
ax.set_title('Cantidad de Transacciones por Clase')
ax.set_xlabel('Clase de Transacción')
ax.set_ylabel('Número de Transacciones')
for p in ax.patches:
    ax.annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', fontsize=11, color='black', xytext=(0, 5),
                textcoords='offset points')
plt.show()

# Gráfico 2: Distribución del valor por clase
plt.figure()
sns.boxplot(x='clase', y='valor', data=df, palette='magma')
plt.title('Distribución del Valor de Transacción por Clase')
plt.xlabel('Clase de Transacción')
plt.ylabel('Valor de la Transacción')
plt.ticklabel_format(style='plain', axis='y') # Evita notación científica
plt.show()
print("----------------------------------------------------------")


# --- 3. Análisis Temporal (fecha_efectiva) ---
print("\n### 3. Análisis Temporal ###")

# Aseguramos que el índice sea la fecha para facilitar el re-muestreo
df_time = df.set_index('fecha_efectiva')

# Re-muestreamos por mes para ver tendencias
# 'M' agrupa por fin de mes
monthly_summary = df_time.resample('M').agg(
    numero_transacciones=('Id_tx', 'count'),
    valor_total=('valor', 'sum')
)

print("\nResumen mensual de transacciones:")
print(monthly_summary)

# Gráfico 3: Tendencia del Volumen de Transacciones Mensuales
plt.figure()
monthly_summary['numero_transacciones'].plot(kind='line', marker='o')
plt.title('Volumen de Transacciones Mensuales')
plt.xlabel('Mes')
plt.ylabel('Número de Transacciones')
plt.ylim(bottom=0) # Asegurar que el eje Y empiece en 0
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.show()

# Gráfico 4: Tendencia del Valor Total Mensual por Clase
plt.figure()
# Pivotamos los datos para tener 'COMPRA' y 'AVANCE' como columnas
valor_por_clase_mes = df_time.groupby([pd.Grouper(freq='M'), 'clase'])['valor'].sum().unstack()

valor_por_clase_mes.plot(kind='line', marker='o', figsize=(12,7))
plt.title('Valor Total Mensual por Clase de Transacción')
plt.xlabel('Mes')
plt.ylabel('Valor Total Transaccionado')
plt.ticklabel_format(style='plain', axis='y')
plt.legend(title='Clase')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.show()

print("\n==========================================================")
print("Análisis Descriptivo Completado ✅")